In [2]:
import json

with open('/Users/wasilhassan/Desktop/sort/credentials.json') as f:
    data = json.load(f)

secret_key = data['mongodb']

print(f"My secret key is {len(secret_key)} characters in lenght.")

My secret key is 70 characters in lenght.


In [3]:
import pymongo
import certifi
import pandas as pd
import re

mongo_connection_string = data['mongodb']

client = pymongo.MongoClient(mongo_connection_string, tlsCAFile=certifi.where())

# Fetch the database named "DA320"
db = client.my_database

# Within the database we have "collections". Think of them as tables in SQL.
allCollections = db.list_collection_names()

# Here is the list of collections within my database
print(f"Using MongoDB version {client.server_info()['version']}.")
print(f"This database has the collections {allCollections}")

Using MongoDB version 6.0.6.
This database has the collections ['1', 'imdb', '2', 'metacritic', 'try_1', 'imdb_view']


In [4]:
#Find data through pipeline view in 2010
db = client['my_database']
view = db['try_1']
data = list(view.find({}))


#convert data into data frame

df = pd.DataFrame(data)
df['release_date'] = pd.to_datetime(df['release_date'])
df = df[df['release_date'].dt.year == 2010]

#Find data in Metacritic in the same year
metacritic_view = pd.DataFrame(db.metacritic.find({ "release_date": re.compile("2010")}))

#convert value to number
metacritic_view.score = pd.to_numeric(metacritic_view.score, errors = 'coerce')
metacritic_view.release_date = pd.to_datetime(metacritic_view.release_date, errors = 'coerce')

print(df)



                            _id       id                      title  runtime  \
15067  6465534b7c96c756573decc0  1245526                        RED    111.0   
15068  6465534b7c96c756573decb6  1375670                  Grown Ups    102.0   
15069  6465534b7c96c756573deccd  1504320          The King's Speech    118.0   
15071  6465534b7c96c756573decbe  1243957                The Tourist    103.0   
15073  6465534b7c96c756573decd2  1386588             The Other Guys    107.0   
...                         ...      ...                        ...      ...   
16436  6465534c7c96c756573df095  1529233  Basilicata Coast to Coast    105.0   
16437  6465534c7c96c756573df0a4  1715730    Y'en aura pas de facile     94.0   
16438  6465534c7c96c756573df0aa  1519245                    Agnosia    105.0   
16439  6465534c7c96c756573df0af  1380201            Run Sister Run!    113.0   
16440  6465534c7c96c756573df0b2  1291125           Au revoir Taipei     85.0   

       user_rating   votes       budget

In [5]:
#Merge the two datasets together using an inner join

unified_view = pd.merge(df, metacritic_view, how="inner", on="title")
unified_view.head()

,_id_x,id,title,runtime,user_rating,votes,budget,opening_weekend,gross_sales,genres,...,company,companyTrimmed,release_date_x,mpaaRating,_id_y,movie_id,release_date_y,description,score,thumbnail
0,6465534b7c96c756573decc0,1245526,RED,111.0,7.0,306196,58000000.0,21761408.0,199006387.0,"[Action, Comedy, Crime, Thriller]",...,"[Summit Entertainment, Di Bonaventura Picture...","[Summit Entertainment, Di Bonaventura Pictures...",2010-09-29,[{'trim': {'input': 'PG-13'}}],6476894895ef4140dc3759a9,15844,2010-10-15,"Frank, Joe, Marvin and Victoria used to be the...",60,https://static.metacritic.com/images/products/...
1,6465534b7c96c756573decb6,1375670,Grown Ups,102.0,5.9,248747,80000000.0,40506562.0,271430189.0,[Comedy],...,"[Columbia Pictures, Relativity Media, Happy ...","[Columbia Pictures, Relativity Media, Happy Ma...",2010-06-24,[{'trim': {'input': 'TV-PG::(TV rating)'}}],6476894895ef4140dc375a74,16047,2010-06-25,"Grown Ups, starring Adam Sandler, Kevin James,...",30,https://static.metacritic.com/images/products/...
2,6465534b7c96c756573deccd,1504320,The King's Speech,118.0,8.0,672909,15000000.0,NaN,373700000.0,"[Biography, Drama, History]",...,"[The Weinstein Company, UK Film Council, Mom...","[The Weinstein Company, UK Film Council, Momen...",2010-09-06,[{'trim': {'input': 'TV-14::(TV rating)'}}],6476894895ef4140dc3758e3,15646,2010-11-26,Based on the true story of the Queen of Englan...,88,https://static.metacritic.com/images/products/...
3,6465534b7c96c756573decbe,1243957,The Tourist,103.0,6.0,240865,100000000.0,16472458.0,278346189.0,"[Action, Thriller]",...,"[GK Films, Columbia Pictures, Spyglass Enter...","[GK Films, Columbia Pictures, Spyglass Enterta...",2010-12-06,[{'trim': {'input': 'PG-13'}}],6476894895ef4140dc375a56,16017,2010-12-10,Johnny Depp stars as an American tourist whose...,37,https://static.metacritic.com/images/products/...
4,6465534b7c96c756573decd2,1386588,The Other Guys,107.0,6.6,264497,100000000.0,35543162.0,170432927.0,"[Action, Comedy, Crime]",...,"[Columbia Pictures, Gary Sanchez Productions,...","[Columbia Pictures, Gary Sanchez Productions, ...",2010-08-02,[{'trim': {'input': 'TV-MA::(TV rating)'}}],6476894895ef4140dc375981,15804,2010-08-06,Two mismatched New York City detectives seize ...,64,https://static.metacritic.com/images/products/...


In [6]:
import transformers
classifier = transformers.pipeline("fill-mask", model="distilbert-base-uncased")

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForMaskedLM: ['activation_13']
- This IS expected if you are initializing TFDistilBertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertForMaskedLM were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForMaskedLM for predictions without further training.


In [7]:
prompt = "movies is [MASK]"
result = classifier(prompt)

for prediction in result:
    print(prediction['sequence'])
    print(prediction['token_str'])
    print(prediction['score'])

movies is!
!
0.1428605616092682
movies is.
.
0.07446049153804779
movies islington
##lington
0.018790876492857933
movies is forever
forever
0.014888285659253597
movies is cool
cool
0.014603724703192711


In [8]:
predictions = []
for _, row in unified_view.iterrows():
    plot_summary = row["description"]

    # Create the prompt to classify the movie as "happy" or "sad"
    prompt = f"This movie is about {plot_summary}. The movie has a [MASK] ending."

    # Use the classifier to predict the sentiment of the movie
    result = classifier(prompt)

    # Get the top predicted sentiment and its score
    predicted_sentiment = result[0]["token_str"]
    sentiment_score = result[0]["score"]

    # Add the prediction to the list
    predictions.append(predicted_sentiment)

# Add the predictions to the dataset as a new column
unified_view["Sentiment"] = predictions

# Print the updated dataset
print(unified_view)

                        _id_x       id  \
0    6465534b7c96c756573decc0  1245526   
1    6465534b7c96c756573decb6  1375670   
2    6465534b7c96c756573deccd  1504320   
3    6465534b7c96c756573decbe  1243957   
4    6465534b7c96c756573decd2  1386588   
..                        ...      ...   
194  6465534c7c96c756573df021  1540814   
195  6465534c7c96c756573df044  1572769   
196  6465534c7c96c756573df04b  1484522   
197  6465534c7c96c756573df082  1568923   
198  6465534c7c96c756573defae   775543   

                                          title  runtime  user_rating   votes  \
0                                           RED    111.0          7.0  306196   
1                                     Grown Ups    102.0          5.9  248747   
2                             The King's Speech    118.0          8.0  672909   
3                                   The Tourist    103.0          6.0  240865   
4                                The Other Guys    107.0          6.6  264497   
..       

In [9]:
import plotly.express as px

unified_view['score'] = unified_view['score'] / 10

# Reshape the data
df_melt = pd.melt(unified_view, id_vars='Sentiment', value_vars=['user_rating', 'score'], 
                  var_name='rating_type', value_name='rating_value')

# Create boxplot
fig = px.box(df_melt, x="Sentiment", y="rating_value", color="rating_type")
fig.show()

In [10]:
#This graph tells the user rating and score of movies with different type of endings. I can deduct from here that movies with happy endings has a median score of 5.2 and a median user rating of 6.3. 


In [11]:
import pandas as pd
import plotly.express as px

# Convert 'release_date_y' to datetime if it's not already
unified_view['release_date_y'] = pd.to_datetime(unified_view['release_date_y'])

# Extract the month and create a new 'Month' column
unified_view['Month'] = unified_view['release_date_y'].dt.month

# Group the data by 'Month' and 'Sentiment' and count the number of sentiments for each month
df_grouped = unified_view.groupby(['Month', 'Sentiment']).size().reset_index(name='Count')

# Create a bar plot
fig = px.bar(df_grouped, x='Month', y='Count', color='Sentiment', barmode='group')
fig.show()


In [12]:
#From this graph I can tell that december was the month when most movies with tragic ending were released, with a total of 18 movies. And June was the month when most movies with happy ending were released, with a total of 9 movies. 

In [13]:
import plotly.express as px

# Create boxplot
fig = px.box(unified_view, x="Sentiment", y="runtime")
fig.show()

In [14]:
#This graph tells us about the runtime of movies with different types of endings. 